In [8]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Remember to install sklearn with
# pip install -U scikit-learn

import sklearn

So now we'll read in the data, and get our X dataframe and Y dataframe

In [10]:
df = pd.read_json('cleaned_train.json')
df.shape

(49041, 16)

In [11]:
df_X = df.drop('interest_level', axis=1)
df_y = df['interest_level']

In [12]:
df_X = df.drop('interest_level', axis=1)
df_X = df_X.drop('description', axis=1)
df_X = df_X.drop('display_address', axis=1)
df_X = df_X.drop('street_address', axis=1)
df_X = df_X.drop('features', axis=1)
df_X = df_X.drop('created', axis=1)
df_X = df_X.drop('listing_id', axis=1)
df_X.shape

(49041, 9)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=123
)


In [23]:
test_df = pd.read_json('test_data_formatted.json')
test_df_unformatted = pd.read_json('test.json')
listing_ids = test_df_unformatted['listing_id']

# Let's do some XGBoosting!

XGBoosting, also known as Extreme Gradient Boosting, is a decision tree ensemble boosting algorithm. 

In [11]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=123
)


In [21]:
xg_clf = xgb.XGBClassifier(
    objective ='reg:linear', 
    colsample_bytree = 0.3, 
    learning_rate = 0.1,
    max_depth = 5, 
    alpha = 10, 
    n_estimators = 10,
)


In [34]:
xg_clf.fit(X_train,y_train)

preds = xg_clf.predict(X_test)
from sklearn.metrics import f1_score

f1_score(y_test, preds, average='weighted')


0.5603588280016875

In [50]:
grid_ans_cv = xg_clf.predict_proba(test_df)
p1 = [ans[0] for ans in grid_ans_cv]
p2 = [ans[1] for ans in grid_ans_cv]
p3 = [ans[2] for ans in grid_ans_cv]
answer_svm_cv = pd.DataFrame({
    "listing_id": listing_ids,
    "high": p1,
    "medium": p2,
    "low": p3
})

In [53]:
answer_svm_cv

,listing_id,high,medium,low
0,7142618,0.193802,0.548337,0.257861
1,7210040,0.195441,0.535764,0.268795
2,7174566,0.190882,0.548426,0.260692
3,7191391,0.193354,0.550843,0.255803
5,7171695,0.195418,0.534066,0.270515
...,...,...,...,...
124003,6928108,0.219833,0.485954,0.294213
124005,6906674,0.194242,0.545874,0.259884
124006,6897967,0.195664,0.537115,0.267222
124007,6842183,0.190146,0.567324,0.242530


In [54]:
# answer_svm_cv.to_csv('ans.csv', index=False)

# Kaggle response: 1.20351 (very bad)

In [82]:
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score, make_scorer

clf_xgb = XGBClassifier()
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }

xg_scorer = make_scorer(f1_score, average='weighted')

clf = RandomizedSearchCV(
    clf_xgb, 
    param_distributions = param_dist, 
    n_iter = 25, 
    scoring = xg_scorer, 
    error_score = 0, 
    verbose = 3, 
    n_jobs = -1
)

numFolds = 5
folds = KFold(n_splits = numFolds, shuffle = True)

X = df_X
y = df_y

estimators = []
results = np.zeros(len(X))
score = 0.0
for train_index, test_index in folds.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()
    clf.fit(X_train, y_train)

    estimators.append(clf.best_estimator_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.9min finished


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.1min finished


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  5.0min finished


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.4min finished


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.9min finished


In [83]:
len(estimators)

5

In [15]:
kclf_0 = estimators[0]

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=123
)

NameError: name 'estimators' is not defined

In [98]:
preds = kclf_0.predict(X_test)

f1_score(y_test, preds, average='micro')

0.8600265062697523

In [80]:
grid_ans_cv = kclf_0.predict_proba(test_df)
p1 = [ans[0] for ans in grid_ans_cv]
p2 = [ans[1] for ans in grid_ans_cv]
p3 = [ans[2] for ans in grid_ans_cv]
answer_svm_cv_folds = pd.DataFrame({
    "listing_id": listing_ids,
    "high": p1,
    "medium": p2,
    "low": p3
})

In [81]:
answer_svm_cv_folds.to_csv('ans_folds.csv', index=False)
# Kaggle give 1.77 => Still very bad :( 

# Gradient Boosting (not extreme)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=123
)

NameError: name 'train_test_split' is not defined

In [104]:
clf = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=1.0,
    max_depth=4, 
    random_state=0
).fit(X_train, y_train)

In [106]:
clf.score(X_test, y_test)

0.7043531450708533

In [13]:
def create_kaggle_response(classifier, csv_name):
    grid_ans_cv = classifier.predict_proba(test_df)
    p1 = [ans[0] for ans in grid_ans_cv]
    p2 = [ans[1] for ans in grid_ans_cv]
    p3 = [ans[2] for ans in grid_ans_cv]
    answer_svm_cv_folds = pd.DataFrame({
        "listing_id": listing_ids,
        "high": p1,
        "medium": p2,
        "low": p3
    })
    answer_svm_cv_folds.to_csv(csv_name, index=False)


In [108]:
# => 2.02 (very bad)

# Simple extreme gradient boosting

In [14]:
import xgboost as xgb # Importing this because I can :) 
from sklearn.metrics import accuracy_score

# Create XGB Classifier object
xgb_clf = xgb.XGBClassifier(
    objective = "multi:softmax" # Multiclass prediction
)

# Fit model
xgb_model = xgb_clf.fit(
    X_train, 
    y_train
)

# Predictions
y_train_preds = xgb_model.predict(X_train)
y_test_preds = xgb_model.predict(X_test)

# Print F1 scores and Accuracy
print("Training F1 Micro Average: ", f1_score(y_train, y_train_preds, average = "micro"))
print("Test F1 Micro Average: ", f1_score(y_test, y_test_preds, average = "micro"))
print("Test Accuracy: ", accuracy_score(y_test, y_test_preds))

NameError: name 'X_train' is not defined

In [141]:
create_kaggle_response(classifier=xgb_clf, csv_name='xgb_simple.csv')
# => 2.02 again

# GPU simple gradient boosting

In [120]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Create XGB Classifier object
xgb_clf = xgb.XGBClassifier(
    tree_method = "gpu_hist", 
    predictor = "gpu_predictor", 
    eval_metric = ["merror", "map", "auc"], 
    objective = "multi:softmax",
)

# Create parameter grid
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000]}

# Create RandomizedSearchCV Object
xgb_rscv = RandomizedSearchCV(
    xgb_clf, 
    param_distributions = parameters, 
    scoring = "f1_micro",                  
    cv = 7, 
    verbose = 3, 
    random_state = 40
)

# Fit the model
model_xgboost = xgb_rscv.fit(X_train, y_train)

Fitting 7 folds for each of 10 candidates, totalling 70 fits
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.701, total=   1.7s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.707, total=   1.4s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.704, total=   1.5s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.704, total=   1.5s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.703, total=   1.5s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=

[CV]  subsample=0.7, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=7, max_depth=4, learning_rate=0.1, gamma=0.1, colsample_bytree=1.0, score=0.715, total=   1.7s
[CV] subsample=0.5, reg_lambda=4.5, reg_alpha=1, n_estimators=500, min_child_weight=7, max_depth=10, learning_rate=0.001, gamma=0.01, colsample_bytree=1.0 
[CV]  subsample=0.5, reg_lambda=4.5, reg_alpha=1, n_estimators=500, min_child_weight=7, max_depth=10, learning_rate=0.001, gamma=0.01, colsample_bytree=1.0, score=0.711, total=  24.3s
[CV] subsample=0.5, reg_lambda=4.5, reg_alpha=1, n_estimators=500, min_child_weight=7, max_depth=10, learning_rate=0.001, gamma=0.01, colsample_bytree=1.0 
[CV]  subsample=0.5, reg_lambda=4.5, reg_alpha=1, n_estimators=500, min_child_weight=7, max_depth=10, learning_rate=0.001, gamma=0.01, colsample_bytree=1.0, score=0.713, total=  24.3s
[CV] subsample=0.5, reg_lambda=4.5, reg_alpha=1, n_estimators=500, min_child_weight=7, max_depth=10, learning_rate=0.001, gamma=0.01, colsampl

[CV]  subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytree=0.6, score=0.723, total=  26.4s
[CV] subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytree=0.6 
[CV]  subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytree=0.6, score=0.720, total=  33.1s
[CV] subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytree=0.6 
[CV]  subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytree=0.6, score=0.715, total=  30.3s
[CV] subsample=0.4, reg_lambda=1.5, reg_alpha=0, n_estimators=1000, min_child_weight=1, max_depth=7, learning_rate=0.01, gamma=0.3, colsample_bytr

[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 11.1min finished


In [142]:
gpu_xgmodel = model_xgboost.best_estimator_

In [143]:
# Predictions
y_train_preds = gpu_xgmodel.predict(X_train)
y_test_preds = gpu_xgmodel.predict(X_test)

# Print F1 scores and Accuracy
print("Training F1 Micro Average: ", f1_score(y_train, y_train_preds, average = "micro"))
print("Test F1 Micro Average: ", f1_score(y_test, y_test_preds, average = "micro"))
print("Test Accuracy: ", accuracy_score(y_test, y_test_preds))

Training F1 Micro Average:  0.7612663132137031
Test F1 Micro Average:  0.7174023855642777
Test Accuracy:  0.7174023855642777


In [144]:
create_kaggle_response(classifier=xgb_clf, csv_name='xgb_gpu.csv')
# => 2.028 again! Ahhh!

# Using teapot

In [17]:
from tpot import TPOTClassifier
import tpot

In [20]:
pipeline_optimizer = TPOTClassifier()


pipeline_optimizer = TPOTClassifier(
    generations=5, 
    population_size=20, 
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=10,
)

pipeline_optimizer.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.7173991882283726
Generation 2 - Current best internal CV score: 0.7173991882283726
Generation 3 - Current best internal CV score: 0.7191069219209947
Generation 4 - Current best internal CV score: 0.7198206711514741
Generation 5 - Current best internal CV score: 0.7198206711514741

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=9, max_features=0.8500000000000001, min_samples_leaf=20, min_samples_split=11, n_estimators=100, subsample=0.9500000000000001)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=10, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=20,
               random_state=42, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [21]:
y_train_preds = pipeline_optimizer.predict(X_train)
y_test_preds = pipeline_optimizer.predict(X_test)

# Print F1 scores and Accuracy
print("Training F1 Micro Average: ", f1_score(y_train, y_train_preds, average = "micro"))
print("Test F1 Micro Average: ", f1_score(y_test, y_test_preds, average = "micro"))
print("Test Accuracy: ", accuracy_score(y_test, y_test_preds))

NameError: name 'f1_score' is not defined

In [24]:
create_kaggle_response(classifier=pipeline_optimizer, csv_name="tpot.csv")